In [0]:
import re
from pyspark.sql import DataFrame

# Utility: Convert camelCase or PascalCase to snake_case
def to_snake_case(name: str) -> str:
    """
    Convert camelCase, PascalCase, or mixed-case strings to snake_case.
    Handles common acronyms like 'ID' → 'id' correctly.
    """
    name = re.sub(r"([a-z0-9])([A-Z])", r"\1_\2", name)  # Insert underscore between camelCase
    name = re.sub(r"([A-Z]+)([A-Z][a-z])", r"\1_\2", name)  # Handle acronyms like IDNumber → ID_Number
    return name.lower()

# Utility: Clean and rename all column names to snake_case using df.toDF()
def clean_and_snake_case_columns(df: DataFrame) -> DataFrame:
    """
    Clean unwanted characters and convert all column names to snake_case.
    """
    cleaned_cols = []
    for col_name in df.columns:
        # Remove unwanted characters and normalize spacing
        cleaned_name = re.sub(r"[ (){};\n\t=]", "", col_name).strip().replace(" ", "_")
        # Convert to snake_case
        snake_case_name = to_snake_case(cleaned_name)
        cleaned_cols.append(snake_case_name)
    return df.toDF(*cleaned_cols)

# Read delta table and clean + snake_case the column names
def read_delta_with_snake_case(spark, path: str) -> DataFrame:
    """
    Read a Delta table and return DataFrame with cleaned snake_case column names.
    """
    df = spark.read.format("delta").load(path)
    return clean_and_snake_case_columns(df)

# Join store and product on store_id
def get_store_product_data(product_df: DataFrame, store_df: DataFrame) -> DataFrame:
    """
    Join product_df and store_df on 'store_id' using inner join.
    """
    return store_df.join(product_df, on="store_id", how="inner")

# Join sales with enriched store-product data on product_id
def enrich_sales_with_store_product(sales_df: DataFrame, store_product_df: DataFrame) -> DataFrame:
    """
    Join sales_df and store_product_df on 'product_id' using inner join.
    """
    return sales_df.join(store_product_df, on="product_id", how="inner")
